In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader


In [13]:
# Defina as transformações para redimensionar as imagens e aplicar transformações de aumento de dados
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalização dos canais RGB
])


In [14]:
# Defina os caminhos para o diretório de treinamento e teste
train_path = r"C:/Users/My/Downloads/Projeto 1/treinamento"
test_path = r"C:/Users/My/Downloads/Projeto 1/teste"

# Carregue os conjuntos de dados de treinamento e teste usando ImageFolder
train_dataset = ImageFolder(train_path, transform=transform)
test_dataset = ImageFolder(test_path, transform=transform)

# Crie os dataloaders para carregar os dados em lotes
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [15]:
# Carregue o modelo ResNet-50 pré-treinado
model = torch.hub.load('pytorch/vision', 'resnet50', pretrained=True)

# Congele os parâmetros do modelo pré-treinado
for param in model.parameters():
    param.requires_grad = False

# Substitua a última camada totalmente conectada para ajustar o número de classes
num_classes = 2
model.fc = nn.Linear(model.fc.in_features, num_classes)


Using cache found in C:\Users\My/.cache\torch\hub\pytorch_vision_main


In [16]:
# Defina a função de perda
criterion = nn.CrossEntropyLoss()

# Defina o otimizador
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)


In [17]:
# Verifique se há suporte para GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Coloque o modelo e os dados no dispositivo
model.to(device)
images = images.to(device)
labels = labels.to(device)

In [18]:
num_epochs = 10

# Coloque o modelo em modo de treinamento
model.train()

# Loop de treinamento
for epoch in range(num_epochs):
    for images, labels in train_loader:
        # Mova as imagens e rótulos para o dispositivo GPU, se disponível
        images = images.to(device)
        labels = labels.to(device)

        # Zere os gradientes dos parâmetros
        optimizer.zero_grad()

        # Faça a passagem direta (forward pass)
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Faça a passagem inversa (backward pass) e otimize os parâmetros
        loss.backward()
        optimizer.step()

    # Imprima a perda a cada época
    print(f'Época [{epoch+1}/{num_epochs}], Perda: {loss.item():.4f}')


C:\ProgramData\anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:858: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Época [1/10], Perda: 0.0015
Época [2/10], Perda: 0.0038
Época [3/10], Perda: 1.5947
Época [4/10], Perda: 0.0015
Época [5/10], Perda: 0.4105
Época [6/10], Perda: 0.0015
Época [7/10], Perda: 0.1721
Época [8/10], Perda: 0.0067
Época [9/10], Perda: 0.0019
Época [10/10], Perda: 0.0008


In [21]:
# Coloque o modelo em modo de avaliação
model.eval()

# Desative o cálculo dos gradientes para economizar memória e acelerar a inferência
with torch.no_grad():
    correct = 0
    total = 0

    # Loop de avaliação
    for images, labels in test_loader:
        # Mova as imagens e rótulos para o dispositivo GPU, se disponível
        images = images.to(device)
        labels = labels.to(device)

        # Faça a passagem direta (forward pass)
        outputs = model(images)

        # Obtenha as previsões do modelo
        _, predicted = torch.max(outputs.data, 1)

        # Atualize o número total de exemplos e o número de previsões corretas
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Calcule a precisão do modelo
    accuracy = 100 * correct / total
    print(f'Acurácia no conjunto de teste: {accuracy:.2f}%')


Acurácia no conjunto de teste: 98.19%
